# AutoML Image Classification Multilabel in pipeline

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Create a pipeline with Image Classification Multilabel AutoML task.

**Motivations** - This notebook explains how to use Image Classification Multilabel AutoML task inside pipeline.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [5]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input, command, Output
from azure.ai.ml.automl import image_classification_multilabel
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities._job.automl.image import ImageClassificationSearchSpace
from azure.ai.ml.sweep import BanditPolicy, Choice, Uniform

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [3]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "gpu-cluster"
print(ml_client.compute.get(cluster_name))

# 2. Basic pipeline job with Image Classification Multilabel task

## 2.1 Build pipeline

In [ ]:
# note that the used docker image doesn't suit for all size of gpu compute. Please use the following command to create gpu compute if experiment failed
# !az ml compute create -n gpu-cluster --type amlcompute --min-instances 0 --max-instances 4 --size Standard_NC12

In [9]:
# Define pipeline
@pipeline(
    description="AutoML Image Clasiification Multilabel Pipeline",
    default_compute="gpu-cluster",
)
def automl_image_classification_miltilabel(
    image_classification_multilabel_train_data,
    image_classification_multilabel_validation_data
):
    # define the automl image_classification_multilabel task with automl function
    image_classification_multilabel_node = image_classification_multilabel(
        training_data=image_classification_multilabel_train_data,
        validation_data=image_classification_multilabel_validation_data,
        target_column_name="label",
        primary_metric="iou",
        properties={"_automl_internal_label": "latest",},
        # currently need to specify outputs "mlflow_model" explictly to reference it in following nodes 
        outputs={"best_model": Output(type="mlflow_model")},
    )
    image_classification_multilabel_node.set_limits(timeout_minutes=120)

    image_classification_multilabel_node.extend_search_space(
        [
            ImageClassificationSearchSpace(
                model_name=Choice(["vitb16r224"]),
                learning_rate=Uniform(0.001, 0.01),
            ),
            ImageClassificationSearchSpace(
                model_name=Choice(["seresnext"]),
                learning_rate=Uniform(0.001, 0.01),
            ),
        ]
    )
    
    image_classification_multilabel_node.set_sweep(
        max_trials=10,
        max_concurrent_trials=2,
        sampling_algorithm="Random",
        early_termination=BanditPolicy(evaluation_interval=2, slack_factor=0.2, delay_evaluation=6),
    )
    
    command_func = command(
        inputs=dict(
            automl_output=Input(type="mlflow_model")
        ),
        command="ls ${{inputs.automl_output}}",
        environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:1"
    )
    show_output = command_func(automl_output=image_classification_multilabel_node.outputs.best_model)


data_folder = "../../automl-standalone-jobs/automl-image-classification-multilabel-task-fridge-items/data"
pipeline = automl_image_classification_miltilabel(
    image_classification_multilabel_train_data=Input(path=f"{data_folder}/training-mltable-folder/", type="mltable"),
    image_classification_multilabel_validation_data=Input(path=f"{data_folder}/validation-mltable-folder/", type="mltable"),
)

# 2.2 Submit pipeline job

In [ ]:
# submit the pipeline job
pipeline_job = ml_client.jobs.create_or_update(
    pipeline, experiment_name="pipeline_samples"
)
pipeline_job

In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

# Next Steps
You can see further examples of running a pipeline job [here](../)